# Withdrawal Strategy Comparison

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/engineerinvestor/financial-health-calculator/blob/main/examples/03_withdrawal_comparison.ipynb)

Compare different withdrawal strategies to find the best approach for your retirement.

In [ ]:
# Install the fundedness package from GitHub
!pip install git+https://github.com/engineerinvestor/financial-health-calculator.git -q

In [ ]:
import numpy as np
import pandas as pd
from fundedness.models.simulation import SimulationConfig
from fundedness.withdrawals.comparison import compare_strategies
from fundedness.withdrawals.fixed_swr import FixedRealSWRPolicy, PercentOfPortfolioPolicy
from fundedness.withdrawals.guardrails import GuardrailsPolicy
from fundedness.withdrawals.vpw import VPWPolicy
from fundedness.withdrawals.rmd_style import RMDStylePolicy
from fundedness.viz.comparison import (
    create_strategy_comparison_chart,
    create_multi_metric_comparison,
)

## 1. Define the Strategies

We'll compare five popular withdrawal approaches:

In [ ]:
# Set up parameters
initial_wealth = 1_000_000
spending_floor = 30_000  # Essential expenses
starting_age = 65

# Create strategies
strategies = [
    FixedRealSWRPolicy(
        withdrawal_rate=0.04,
        floor_spending=spending_floor,
    ),
    PercentOfPortfolioPolicy(
        withdrawal_rate=0.04,
        floor=spending_floor,
    ),
    GuardrailsPolicy(
        initial_rate=0.05,
        upper_guardrail=0.06,
        lower_guardrail=0.04,
        floor_spending=spending_floor,
    ),
    VPWPolicy(
        starting_age=starting_age,
        floor_spending=spending_floor,
    ),
    RMDStylePolicy(
        starting_age=starting_age,
        floor_spending=spending_floor,
    ),
]

for s in strategies:
    print(f"- {s.name}: {s.description}")

## 2. Run the Comparison

In [ ]:
%%time
# Run comparison with same random seed for fair comparison
config = SimulationConfig(
    n_simulations=5000,
    n_years=30,
    random_seed=42,
)

comparison = compare_strategies(
    policies=strategies,
    initial_wealth=initial_wealth,
    config=config,
    stock_weight=0.6,
    starting_age=starting_age,
    spending_floor=spending_floor,
)

## 3. Compare Key Metrics

In [ ]:
# Create metrics table
metrics_data = []
for name, metrics in comparison.metrics.items():
    metrics_data.append({
        "Strategy": name,
        "Success Rate": f"{metrics['success_rate']:.1%}",
        "Floor Breach": f"{metrics['floor_breach_rate']:.1%}",
        "Median Terminal": f"${metrics['median_terminal_wealth']:,.0f}",
        "Initial Spending": f"${metrics['median_initial_spending']:,.0f}",
        "Avg Spending": f"${metrics['average_spending']:,.0f}",
        "Volatility": f"{metrics['spending_volatility']:.1%}",
    })

df = pd.DataFrame(metrics_data)
df

## 4. Wealth Paths Comparison

In [ ]:
years = np.arange(1, config.n_years + 1)

# Prepare data for comparison chart
wealth_data = {}
for name, result in comparison.results.items():
    wealth_data[name] = {
        "wealth_median": result.wealth_percentiles.get("P50", np.zeros(config.n_years))
    }

fig = create_strategy_comparison_chart(
    years=years,
    strategies=wealth_data,
    metric="wealth_median",
    title="Median Wealth Over Time by Strategy",
    y_label="Portfolio Value ($)",
)
fig.show()

## 5. Spending Paths Comparison

In [ ]:
spending_data = {}
for name, result in comparison.results.items():
    spending_data[name] = {
        "spending_median": result.spending_percentiles.get("P50", np.zeros(config.n_years))
    }

fig = create_strategy_comparison_chart(
    years=years,
    strategies=spending_data,
    metric="spending_median",
    title="Median Spending Over Time by Strategy",
    y_label="Annual Spending ($)",
)

# Add floor line
fig.add_hline(y=spending_floor, line_dash="dash", line_color="red",
              annotation_text="Spending Floor")
fig.show()

## 6. Multi-Metric Dashboard

In [ ]:
# Prepare multi-metric data
multi_data = {}
for name, result in comparison.results.items():
    multi_data[name] = {
        "wealth_median": result.wealth_percentiles.get("P50", np.zeros(config.n_years)),
        "spending_median": result.spending_percentiles.get("P50", np.zeros(config.n_years)),
        "survival_prob": result.get_survival_probability(),
    }

fig = create_multi_metric_comparison(
    years=years,
    strategies=multi_data,
    title="Strategy Comparison Dashboard",
)
fig.show()

## 7. Trade-off Analysis

Plot success rate vs average spending to see the trade-off:

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

for name, metrics in comparison.metrics.items():
    fig.add_trace(go.Scatter(
        x=[metrics['average_spending']],
        y=[metrics['success_rate'] * 100],
        mode='markers+text',
        text=[name],
        textposition='top center',
        marker=dict(size=15),
        name=name,
    ))

fig.add_hline(y=90, line_dash="dash", line_color="gray",
              annotation_text="90% success target")

fig.update_layout(
    title="Success Rate vs Average Spending",
    xaxis_title="Average Annual Spending ($)",
    yaxis_title="Success Rate (%)",
    template="plotly_white",
    showlegend=False,
)
fig.show()

## 8. Strategy Recommendations

Based on the comparison:

In [ ]:
# Find best strategies for different goals
metrics = comparison.metrics

highest_success = max(metrics.items(), key=lambda x: x[1]['success_rate'])
highest_spending = max(metrics.items(), key=lambda x: x[1]['average_spending'])
lowest_volatility = min(metrics.items(), key=lambda x: x[1]['spending_volatility'])
highest_terminal = max(metrics.items(), key=lambda x: x[1]['median_terminal_wealth'])

print("Strategy Recommendations:")
print(f"")
print(f"For MAXIMUM SAFETY: {highest_success[0]}")
print(f"  - Success rate: {highest_success[1]['success_rate']:.1%}")
print(f"")
print(f"For HIGHEST SPENDING: {highest_spending[0]}")
print(f"  - Average spending: ${highest_spending[1]['average_spending']:,.0f}/year")
print(f"")
print(f"For STABLE SPENDING: {lowest_volatility[0]}")
print(f"  - Spending volatility: {lowest_volatility[1]['spending_volatility']:.1%}")
print(f"")
print(f"For MAXIMUM LEGACY: {highest_terminal[0]}")
print(f"  - Median terminal wealth: ${highest_terminal[1]['median_terminal_wealth']:,.0f}")

## Summary

Key findings:

1. **Fixed SWR** provides stable, predictable spending but may be too conservative
2. **Percent of Portfolio** adapts to markets but creates volatile spending
3. **Guardrails** balances stability with adaptability
4. **VPW** increases spending as you age, reducing legacy
5. **RMD-Style** similar to VPW with different rate progression

Choose based on your priorities: safety, spending level, stability, or legacy!